# Notebook 5: Predictions with the Best Regression Model

In this notebook we take the supervised learning setup from Notebook 4 and go one step further:

- we train and evaluate three regression models (Linear Regression, KNN Regressor, Random Forest Regressor);
- we select the best model based on the lowest test RMSE;
- we refit the best model on all available data and use it to make predicted popularity scores for every song;
- we save these predictions in a new file `spotify_with_predictions.csv`.

In the next notebook we will use this file to better understand which features drive high popularity.

## 1. Load the modeling dataset and define X and y

We start from `spotify_model_df.csv` created in Notebook 2. We separate the feature matrix `X` from the target variable `y = track_popularity`.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

pd.set_option('display.max_columns', 100)

spotify_model_df = pd.read_csv('spotify_model_df.csv')
spotify_model_df.head()

,danceability,energy,speechiness,instrumentalness,liveness,valence,tempo,duration_min,release_year,key,mode,time_signature,playlist_genre,track_popularity,label_kaggle
0,0.521,0.592,0.0304,0.0000,0.122,0.535,157.969,4.194467,2024.0,6.0,0.0,3.0,pop,100,1
1,0.747,0.507,0.0358,0.0608,0.117,0.438,104.978,3.506217,2024.0,2.0,1.0,4.0,pop,97,1
2,0.554,0.808,0.0368,0.0000,0.159,0.372,108.548,2.771667,2024.0,1.0,1.0,4.0,pop,93,1
3,0.670,0.910,0.0634,0.0000,0.304,0.786,112.966,2.621333,2024.0,0.0,0.0,4.0,pop,81,1
4,0.777,0.783,0.2600,0.0000,0.355,0.939,149.027,2.831950,2024.0,0.0,0.0,4.0,pop,98,1


In [2]:
# Define target y
if 'track_popularity' not in spotify_model_df.columns:
    raise ValueError('Column track_popularity is missing from the dataset.')

y = spotify_model_df['track_popularity']

# X contains all features except the target and label_kaggle (if present)
X = spotify_model_df.drop(columns=[c for c in ['track_popularity', 'label_kaggle']
                                   if c in spotify_model_df.columns])
X.head()

,danceability,energy,speechiness,instrumentalness,liveness,valence,tempo,duration_min,release_year,key,mode,time_signature,playlist_genre
0,0.521,0.592,0.0304,0.0000,0.122,0.535,157.969,4.194467,2024.0,6.0,0.0,3.0,pop
1,0.747,0.507,0.0358,0.0608,0.117,0.438,104.978,3.506217,2024.0,2.0,1.0,4.0,pop
2,0.554,0.808,0.0368,0.0000,0.159,0.372,108.548,2.771667,2024.0,1.0,1.0,4.0,pop
3,0.670,0.910,0.0634,0.0000,0.304,0.786,112.966,2.621333,2024.0,0.0,0.0,4.0,pop
4,0.777,0.783,0.2600,0.0000,0.355,0.939,149.027,2.831950,2024.0,0.0,0.0,4.0,pop


## 2. Define numerical and categorical feature lists

We use the same logic as in Notebook 4, this is kind of a repetition but it makes this notebook more coherent and it makes it easier to understand the full logic. Numerical features include the main audio variables and the engineered features `duration_min` and `release_year`. Categorical features include musical categories like `key`, `mode`, `time_signature`, and `playlist_genre` (if present).

In [3]:
# List of possible numerical audio features
audio_numeric_features = [
    'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo'
]

numeric_features = [f for f in audio_numeric_features if f in X.columns]

# Add engineered numerical features
for extra in ['duration_min', 'release_year']:
    if extra in X.columns:
        numeric_features.append(extra)

categorical_features = []
for col in ['key', 'mode', 'time_signature', 'playlist_genre']:
    if col in X.columns:
        categorical_features.append(col)

print('Numerical features:', numeric_features)
print('Categorical features:', categorical_features)

Numerical features: ['danceability', 'energy', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_min', 'release_year']
Categorical features: ['key', 'mode', 'time_signature', 'playlist_genre']


## 3. Preprocessing pipeline

We reuse the same preprocessing strategy as before:

- numerical features: impute missing values with the **median**, then standardize with `StandardScaler`;
- categorical features: impute missing values with the **most frequent** value, then apply one-hot encoding.

We combine these steps using a `ColumnTransformer`.

In [4]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

## 4. Train/test split

We create a training set (80%) and a test set (20%). We will use the training set to fit the models and the test set to estimate how well they generalize to new songs.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

X_train.shape, X_test.shape

((3864, 13), (966, 13))

## 5. Define models and full pipelines

We build one pipeline for each model: the pipeline first applies the preprocessing, then fits the regression model.

In [6]:
# 1) Linear Regression
linreg_model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', LinearRegression())
])

# 2) KNN Regressor (with k=5, we choose k=5 as a common default value as instructed, 
# of course this parameter could be better identified with hyperparameter tuning but we tried to not over complicate things)
knn_model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=5))
])

# 3) Random Forest Regressor
rf_model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=0))
])

models = {
    'LinearRegression': linreg_model,
    'KNNRegressor': knn_model,
    'RandomForestRegressor': rf_model
}

## 6. Evaluation function and model comparison

We define a helper function that fits a model, then computes MSE, RMSE, MAE and R² on both train and test sets. We then compare all models in a single table and select the one with the **lowest test RMSE**.

In [7]:
def evaluate_regression_model(name, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    
    rmse_train = np.sqrt(mse_train)
    rmse_test = np.sqrt(mse_test)
    
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    
    return {
        'model': name,
        'mse_train': mse_train,
        'mse_test': mse_test,
        'rmse_train': rmse_train,
        'rmse_test': rmse_test,
        'mae_train': mae_train,
        'mae_test': mae_test,
        'r2_train': r2_train,
        'r2_test': r2_test
    }

results = []
for name, model in models.items():
    results.append(evaluate_regression_model(name, model,
                                             X_train, X_test, y_train, y_test))

results_df = pd.DataFrame(results)
results_df.sort_values(by='rmse_test')

,model,mse_train,mse_test,rmse_train,rmse_test,mae_train,mae_test,r2_train,r2_test
2,RandomForestRegressor,29.290002,193.690540,5.412024,13.917275,3.993330,10.263686,0.926282,0.485101
0,LinearRegression,268.725073,256.318596,16.392836,16.009953,12.640921,12.420396,0.323668,0.318613
1,KNNRegressor,199.387236,275.177226,14.120455,16.588467,10.561439,12.223602,0.498179,0.268479


The model with the **lowest `rmse_test`** is the one that, on average, makes the smallest error in predicting popularity (in points on the 0–100 scale) on unseen data.

In [8]:
from sklearn.model_selection import GridSearchCV

# Rebuild a RF pipeline compatible with GridSearch
rf_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', RandomForestRegressor(random_state=0))
])

param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [None, 10, 20]
}

grid_search = GridSearchCV(
    rf_pipeline,
    param_grid=param_grid,
    cv=3,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", -grid_search.best_score_)


Best parameters: {'model__max_depth': None, 'model__n_estimators': 200}
Best CV RMSE: 14.88597497547255


In [9]:
best_rf_tuned = grid_search.best_estimator_

y_test_pred_tuned = best_rf_tuned.predict(X_test)

mse_test_tuned = mean_squared_error(y_test, y_test_pred_tuned)
rmse_test_tuned = np.sqrt(mse_test_tuned)
mae_test_tuned = mean_absolute_error(y_test, y_test_pred_tuned)
r2_test_tuned = r2_score(y_test, y_test_pred_tuned)

print("Tuned RF – test RMSE:", rmse_test_tuned)
print("Tuned RF – test MAE:", mae_test_tuned)
print("Tuned RF – test R²:", r2_test_tuned)


Tuned RF – test RMSE: 13.863292235763735
Tuned RF – test MAE: 10.207189095928227
Tuned RF – test R²: 0.4890871893402142


Again, untill this point we basically repeated the information and steps from Notebook 4. Again this was done purely academic purpose to make this notebook more coherent and easier to understand the full logic we used.

## 7. Select the best model and refit on all data

Now we select the model with the lowest test RMSE. To make predictions for **all songs**, we refit this best model on the full dataset `(X, y)` (not only on the training split). This way the model uses all available information before producing final predictions.

In [10]:
# Select the best model according to test RMSE
best_row = results_df.sort_values(by='rmse_test').iloc[0]
best_name = best_row['model']
print('Best model based on test RMSE:', best_name)

# Refit the tuned RF on ALL data (X, y)
best_rf_tuned.fit(X, y)

# Predict popularity for all songs using the tuned model
y_pred_all = best_rf_tuned.predict(X)



Best model based on test RMSE: RandomForestRegressor


## 8. Add predictions to the dataset and inspect results

We create a new DataFrame with an extra column `predicted_popularity`, which contains the model's prediction for each song. We also look at a few simple diagnostics, such as correlation between true and predicted popularity and a quick summary of the prediction errors.

In [11]:
spotify_with_predictions = spotify_model_df.copy()
spotify_with_predictions['predicted_popularity'] = y_pred_all

spotify_with_predictions[['track_popularity', 'predicted_popularity']].head()

,track_popularity,predicted_popularity
0,100,95.413333
1,97,92.635000
2,93,90.975000
3,81,82.555000
4,98,92.740000


In [12]:
# Correlation between true and predicted popularity
spotify_with_predictions[['track_popularity', 'predicted_popularity']].corr()

,track_popularity,predicted_popularity
track_popularity,1.000000,0.976622
predicted_popularity,0.976622,1.000000


In [13]:
# Simple error analysis
errors = spotify_with_predictions['predicted_popularity'] - spotify_with_predictions['track_popularity']
errors.describe()

count    4830.000000
mean        0.038305
std         5.202415
min       -20.870000
25%        -2.963750
50%        -0.345000
75%         2.382500
max        22.395000
dtype: float64

## 9. Save the dataset with predictions

Finally, we save the augmented dataset as `spotify_with_predictions.csv`. In the next notebook we will use this file to create a binary popularity label and study which features are most influential for predicting whether a song is popular.

In [14]:
spotify_with_predictions.to_csv('spotify_with_predictions.csv', index=False)
spotify_with_predictions.head()

,danceability,energy,speechiness,instrumentalness,liveness,valence,tempo,duration_min,release_year,key,mode,time_signature,playlist_genre,track_popularity,label_kaggle,predicted_popularity
0,0.521,0.592,0.0304,0.0000,0.122,0.535,157.969,4.194467,2024.0,6.0,0.0,3.0,pop,100,1,95.413333
1,0.747,0.507,0.0358,0.0608,0.117,0.438,104.978,3.506217,2024.0,2.0,1.0,4.0,pop,97,1,92.635000
2,0.554,0.808,0.0368,0.0000,0.159,0.372,108.548,2.771667,2024.0,1.0,1.0,4.0,pop,93,1,90.975000
3,0.670,0.910,0.0634,0.0000,0.304,0.786,112.966,2.621333,2024.0,0.0,0.0,4.0,pop,81,1,82.555000
4,0.777,0.783,0.2600,0.0000,0.355,0.939,149.027,2.831950,2024.0,0.0,0.0,4.0,pop,98,1,92.740000
